# Standford/MIT lab dataset

In [ ]:
import warnings
from time import time
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io
from os import listdir

In [ ]:
%matplotlib qt
"""General parameters applied as default parameters to the majority of the following functions"""
# plt.rcParams["figure.figsize"] = (16/2.54, 16/2.54)
plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams["axes.grid"] = True
plt.rcParams["font.size"] = 11
from cycler import cycler
colors = cycler(
    "color",[
        "#00549F", # 100% blue
        "#000000", # black
        "#73BDFF", # 40% blue
        "#990516", # 100% red
        "#B97E00", # 100% yellow
        "#C8C8C8", # 20% gray
        "#FFDE95", # 20% yellow
        "#FDC5CC", # 20% red
        "#2C9CFF", # 60% blue
        "#F95265", # 60% red
        "#F6A800", # 60% yellow
    ],
)
plt.rc("axes", facecolor="w", axisbelow=True, grid=True, prop_cycle=colors)
plt.rcParams["font.family"] = "Arial"
# plt.rc("grid", color="k", linestyle="solid", alpha =0.5)
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

In [ ]:
warnings.filterwarnings("ignore")
import sys
sys.setrecursionlimit(10000)
pd.set_option('display.max_columns', None)

path = "/Users/kenny/Library/CloudStorage/OneDrive-ACCUREBatteryIntelligenceGmbH/Thesis/data/StanfordMIT"

## Data Preprocessing

In [ ]:
combine = pd.read_parquet(f"{path}/combined.parquet")

In [ ]:
upper = 112
lower = 100
combine = pd.DataFrame()
for f in listdir(path=path):
    if f.endswith(".csv"):
        name = f[0:f.index(".csv")]
        file = pd.read_csv(f"{path}/{f}")
        meta = pd.read_csv(f"{path}/metadata/{name}_Metadata.csv")
        file["soh"] = file["Discharge_Capacity"].rolling(window=2500).max()*100
        start = file[(file["soh"]>lower) & (file["soh"]<upper)].index[1000]
        file = file.iloc[start:]
        file = file[file["soh"]<upper]
        file["test_id"] = meta["test_id"][0]
        file["device_id"] = meta["device_id"][0]
        combine = pd.concat([combine,file],ignore_index=True)
combine = combine.drop("Aux_Voltage",axis=1)
combine.to_parquet(f"{path}/combined.parquet")
combine

In [ ]:
# check the voltage distribution and choose the upper and lower threshold
combine["Voltage"].hist()
print(combine["Voltage"].quantile(0.75))
print(combine["Voltage"].quantile(0.2))

In [ ]:
# same charge curve
file.loc[file["Cycle_Index"]==20,"Discharge_Capacity"].plot()

In [ ]:
# input feature extraction
upper = 112
lower = 100
step = 1
volt_threshold = 3.3
train = pd.DataFrame()
for f in listdir(path=path):
    if f.endswith(".csv"):
        df = pd.DataFrame()
        name = f[0:f.index(".csv")]
        print(name)
        meta = pd.read_csv(f"{path}/metadata/{name}_Metadata.csv")
        file = pd.read_csv(f"{path}/{f}")
        file["soh"] = file["Discharge_Capacity"].rolling(window=2500).max()*100
        start = file[(file["soh"]>lower) & (file["soh"]<upper)].index[1000]
        file = file.iloc[start:]
        file = file[file["soh"]<upper]
        file["day"] = (file["DateTime"]-file["DateTime"].iloc[0])/60/60/24
        df["day"] = np.arange(int(file["day"].min())+1,int(file["day"].max())+1,step)
        df["test_id"] = meta["test_id"][0]
        df["device_id"] = meta["device_id"][0]
        for i in df["day"]:
            data = file[(file["day"]>=i-1) & (file["day"]<i)]
            df.loc[df["day"]==i,"soh"] = data["soh"].median()
            df.loc[df["day"]==i,"dsoh"] = data["soh"].quantile(0.01)-data["soh"].quantile(0.99)
            df.loc[df["day"]==i,"temp_mean"] = data["Temperature"].mean()
            df.loc[df["day"]==i,"temp_98q"] = data["Temperature"].quantile(0.98)
            df.loc[df["day"]==i,"temp_2q"] = data["Temperature"].quantile(0.02)
            df.loc[df["day"]==i,"volt_mean"] = data["Voltage"].mean()
            df.loc[df["day"]==i,"volt_98q"] = data["Voltage"].quantile(0.98)
            df.loc[df["day"]==i,"volt_2q"] = data["Voltage"].quantile(0.02)
            df.loc[df["day"]==i,"volt_over"] = data[data["Voltage"]>volt_threshold]["Voltage"].count()
            df.loc[df["day"]==i,"curr_mean"] = data["Current"].mean()
            df.loc[df["day"]==i,"curr_mean_chg"] = data[data["Current"]>0]["Current"].mean()
            df.loc[df["day"]==i,"curr_mean_dsc"] = data[data["Current"]<0]["Current"].mean()
            df.loc[df["day"]==i,"curr_use_chg"] = data[data["Current"]>0.5]["Current"].mean()
            df.loc[df["day"]==i,"curr_use_dsc"] = data[data["Current"]<-0.5]["Current"].mean()
            df.loc[df["day"]==i,"curr_98q"] = data["Current"].quantile(0.98)
            df.loc[df["day"]==i,"curr_2q"] = data["Current"].quantile(0.02)
            pow_chg = (data[data["Current"]>0]["Voltage"]*data[data["Current"]>0]["Current"]).mean()
            pow_dsg = (data[data["Current"]<0]["Voltage"]*data[data["Current"]<0]["Current"]).mean()
            df.loc[df["day"]==i,"power_charge_mean"] = pow_chg
            df.loc[df["day"]==i,"power_discharge_mean"] = pow_dsg
            df.loc[df["day"]==i,"dod"] = data["Cycle_Index"].max()-data["Cycle_Index"].min()
        train = pd.concat([train,df],ignore_index=True)
train.to_parquet(f"{path}/training.parquet")
train

In [ ]:
train[train["test_id"]==13]["soh"].plot()

## Load Data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import sklearn.metrics as metrics
from accure_analytics.utils.error_metrics import mean_squared_error as rms
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [ ]:
df = pd.read_parquet(f"{path}/training.parquet").dropna()

In [ ]:
ids = df["test_id"].unique()
np.random.seed(200)
verif_id = np.random.choice(ids,size=5)
verif = df[df["test_id"].isin(verif_id)]
x_verif = verif.drop(["test_id","dsoh","soh"],axis=1)
y_verif = verif["dsoh"].values
# train_set = df[~df["test_id"].isin(verif_id)]
train_set = df
x = train_set.drop(["test_id","dsoh","soh"],axis=1)
y = train_set["dsoh"].values
shuffle_set = train_set.sample(frac=1, random_state=200)
x_shuffle = shuffle_set.drop(["test_id","dsoh","soh"],axis=1)
y_shuffle = shuffle_set["dsoh"].values
x_scaled = sc.fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(
    x_scaled, y, test_size=0.2, random_state=200)


## Visualization

In [ ]:
df

In [ ]:
feat = df.drop(["test_id","dsoh","soh","device_id"],axis=1)
y = df['dsoh']
fig, axs = plt.subplots(6,3,figsize=(8,20),constrained_layout=True)
fig.supylabel("∆Q")
fig.suptitle("Dataset Delta Scatter Plot")
for i in range(6):
    for j in range(3):
        n = 3*i+j
        if n >= 6*3:
            break
        col = feat.columns[n]
        axs[i,j].scatter(feat[col],y,c="#00549F",s=1.5,label=col)
        axs[i,j].set_xlabel(col)
plt.savefig("/Users/kenny/Library/CloudStorage/OneDrive-ACCUREBatteryIntelligenceGmbH/Thesis/figures/corr_lab.svg")

## Tuning

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge 
ridge = Ridge(alpha=0)
ridge.fit(x_train,y_train)
pred = ridge.predict(x_test)
print('RMSE: %.4f'%rms(y_test,pred))
print('R2: %.4f'%metrics.r2_score(y_test,pred))

In [ ]:
from sklearn.linear_model import Lasso 
lasso = Lasso(alpha=0)
lasso.fit(x_train,y_train)
pred = lasso.predict(x_test)
print('RMSE: %.4f'%rms(y_test,pred))
print('R2: %.4f'%metrics.r2_score(y_test,pred))

In [ ]:
from sklearn.linear_model import ElasticNet 
en = ElasticNet(alpha=0.,l1_ratio=0.)
en.fit(x_train,y_train)
pred = en.predict(x_test)
print('RMSE: %.4f'%rms(y_test,pred))
print('R2: %.4f'%metrics.r2_score(y_test,pred))

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
poly_feat = PolynomialFeatures(degree=6,include_bias=False)
poly = make_pipeline(poly_feat,LinearRegression())
poly.fit(x_train,y_train)
pred = poly.predict(x_test)
print('RMSE: %.4f'%rms(y_test,pred))
print('R2: %.4f'%metrics.r2_score(y_test,pred))

In [ ]:

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process import kernels
gpr = GaussianProcessRegressor(kernel=kernels.Matern())
# gpr = GaussianProcessRegressor(kernel=kernels.RationalQuadratic())
gpr.fit(x_train,y_train)
pred = gpr.predict(x_test)
print('RMSE: %.4f'%rms(y_test,pred))
print('R2: %.4f'%metrics.r2_score(y_test,pred))

In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
# svr = SVR(kernel='rbf',epsilon=0.03,C=8)
svr = SVR(kernel='linear',epsilon=0.01,C=8)
svr.fit(x_train,y_train)
pred = svr.predict(x_test)
print('RMSE: %.4f'%rms(y_test,pred))
print('R2: %.4f'%metrics.r2_score(y_test,pred))


In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
pcr = make_pipeline(PCA(n_components=12), LinearRegression())

pcr.fit(x_train, y_train)
pred = pcr.predict(x_test) 
print('RMSE: %.4f'%rms(y_test,pred))
print('R2: %.4f'%metrics.r2_score(y_test,pred))


In [ ]:

from sklearn.cross_decomposition import PLSRegression
pls = PLSRegression(n_components=25)
pls.fit(x_train,y_train)
pred = pls.predict(x_test).flatten()
print('RMSE: %.4f'%rms(y_test,pred))
print('R2: %.4f'%metrics.r2_score(y_test,pred))


In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_validate
nn1 = MLPRegressor(hidden_layer_sizes=(50,50,50,50),activation='relu',alpha=0.1,learning_rate='constant',random_state=100)
nn1.fit(x_train, y_train)
pred = nn1.predict(x_test) 
print('RMSE: %.4f'%rms(y_test,pred))
print('R2: %.4f'%metrics.r2_score(y_test,pred))


In [ ]:

nn3 = MLPRegressor(hidden_layer_sizes=(200,200,100,100),activation='relu',alpha=0.07,learning_rate='constant',random_state=200)
nn3.fit(x_train, y_train)
pred = nn3.predict(x_test) 
print('RMSE: %.4f'%rms(y_test,pred))
print('R2: %.4f'%metrics.r2_score(y_test,pred))

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'hidden_layer_sizes': [
    (200,200,100,100)],
    'alpha':[0.07,0.1,0.05],
    'learning_rate':['adaptive','constant']
    }  
grid = GridSearchCV(MLPRegressor(), param_grid, cv=3, verbose=3) 
grid.fit(x_train, y_train)
print(grid.best_params_) 

In [ ]:
print(grid.best_params_) 

In [ ]:
svr = SVR(kernel='rbf',epsilon=0.03,C=8)
svr.fit(x_train,y_train)
pred = svr.predict(x_test)
print('RMSE',rms(y_test,pred))
print('R2',metrics.r2_score(y_test,pred))

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
poly_feat = PolynomialFeatures(degree=2,include_bias=False)
poly = make_pipeline(poly_feat,LinearRegression())
poly.fit(x_train,y_train)
pred = poly.predict(x_test)
print('RMSE',rms(y_test,pred))
print('R2',metrics.r2_score(y_test,pred))

## Models

In [ ]:
train

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
lr = LinearRegression()
lr.fit(x_train,y_train)
pred = lr.predict(x_test)
scores = cross_validate(lr,x_shuffle,y_shuffle,scoring='r2')
stats = pd.DataFrame(scores).describe()
stats.test_score

In [ ]:
# coefficient analysis from LR
for i,v in enumerate(lr.coef_):
    print("Feature: ",x.columns[i],"=",v)
plt.figure(figsize=(8,5),tight_layout=True)
plt.xticks(rotation=45, ha="right")
plt.bar(x.columns,np.abs(lr.coef_),color="#00549F")
plt.ylabel("Coefficient Magnitude")
plt.title("Input Feature Correlations from Dataset Delta")
plt.savefig("/Users/kenny/Library/CloudStorage/OneDrive-ACCUREBatteryIntelligenceGmbH/Thesis/figures/coef_lab.svg")

In [ ]:
# collect performance metrics
from time import time
from sklearn.model_selection import cross_validate
result = pd.DataFrame()
cv = 4

from sklearn.linear_model import LinearRegression
lr = LinearRegression(normalize=True)
t0 = time()
lr.fit(x_train,y_train)
pred = lr.predict(x_test)
t1 = time()
stats = pd.DataFrame(cross_validate(lr,x_shuffle,y_shuffle,cv=cv,scoring='r2')).describe()
result = result.append({'model':'Linear','runtime':(t1-t0),'RMSE':rms(y_test,pred),
    'R2':metrics.r2_score(y_test,pred),'CV_runtime':stats.fit_time[1]+stats.score_time[1],
    'CV_score':stats.test_score[1],'CV_std':stats.test_score[2]},ignore_index=True)

from sklearn.linear_model import Ridge 
ridge = Ridge(alpha=0.5)
t0 = time()
ridge.fit(x_train,y_train)
pred = ridge.predict(x_test)
t1 = time()
stats = pd.DataFrame(cross_validate(ridge,x_shuffle,y_shuffle,cv=cv)).describe()
result = result.append({'model':'Ridge','runtime':(t1-t0),'RMSE':rms(y_test,pred),
    'R2':metrics.r2_score(y_test,pred),'CV_runtime':stats.fit_time[1]+stats.score_time[1],
    'CV_score':stats.test_score[1],'CV_std':stats.test_score[2]},ignore_index=True)

from sklearn.linear_model import Lasso 
lasso = Lasso(alpha=0.)
t0 = time()
lasso.fit(x_train,y_train)
pred = lasso.predict(x_test)
t1 = time()
stats = pd.DataFrame(cross_validate(lasso,x_shuffle,y_shuffle,cv=cv)).describe()
result = result.append({'model':'Lasso','runtime':(t1-t0),'RMSE':rms(y_test,pred),
    'R2':metrics.r2_score(y_test,pred),'CV_runtime':stats.fit_time[1]+stats.score_time[1],
    'CV_score':stats.test_score[1],'CV_std':stats.test_score[2]},ignore_index=True)

from sklearn.linear_model import ElasticNet 
en = ElasticNet(alpha=0.)
t0 = time()
en.fit(x_train,y_train)
pred = en.predict(x_test)
t1 = time()
stats = pd.DataFrame(cross_validate(en,x_shuffle,y_shuffle,cv=cv)).describe()
result = result.append({'model':'ElasticNet','runtime':(t1-t0),'RMSE':rms(y_test,pred),
    'R2':metrics.r2_score(y_test,pred),'CV_runtime':stats.fit_time[1]+stats.score_time[1],
    'CV_score':stats.test_score[1],'CV_std':stats.test_score[2]},ignore_index=True)

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
poly_feat = PolynomialFeatures(degree=2,include_bias=False)
poly = make_pipeline(poly_feat,LinearRegression())
poly.fit(x_train,y_train)
pred = poly.predict(x_test)
stats = pd.DataFrame(cross_validate(poly,x_shuffle,y_shuffle,cv=cv)).describe()
result = result.append({'model':'Poly','runtime':(t1-t0),'RMSE':rms(y_test,pred),
    'R2':metrics.r2_score(y_test,pred),'CV_runtime':stats.fit_time[1]+stats.score_time[1],
    'CV_score':stats.test_score[1],'CV_std':stats.test_score[2]},ignore_index=True)

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process import kernels
gpr = GaussianProcessRegressor(kernel=kernels.RationalQuadratic())
t0 = time()
gpr.fit(x_train,y_train)
pred = gpr.predict(x_test)
t1 = time()
stats = pd.DataFrame(cross_validate(gpr,x_shuffle,y_shuffle,cv=cv)).describe()
result = result.append({'model':'GPR','runtime':(t1-t0),'RMSE':rms(y_test,pred),
    'R2':metrics.r2_score(y_test,pred),'CV_runtime':stats.fit_time[1]+stats.score_time[1],
    'CV_score':stats.test_score[1],'CV_std':stats.test_score[2]},ignore_index=True)

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
svr = SVR(kernel='rbf',epsilon=0.03,C=8)
t0 = time()
svr.fit(x_train,y_train)
pred = svr.predict(x_test)
t1 = time()
stats = pd.DataFrame(cross_validate(svr,x_shuffle,y_shuffle,cv=cv)).describe()
result = result.append({'model':'SVR','runtime':(t1-t0),'RMSE':rms(y_test,pred),
    'R2':metrics.r2_score(y_test,pred),'CV_runtime':stats.fit_time[1]+stats.score_time[1],
    'CV_score':stats.test_score[1],'CV_std':stats.test_score[2]},ignore_index=True)

from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
pcr = make_pipeline(PCA(n_components=19), LinearRegression())
t0 = time()
pcr.fit(x_train, y_train)
pred = pcr.predict(x_test) 
t1 = time()
stats = pd.DataFrame(cross_validate(pcr,x_shuffle,y_shuffle,cv=cv)).describe()
result = result.append({'model':'PCA','runtime':(t1-t0),'RMSE':rms(y_test,pred),
    'R2':metrics.r2_score(y_test,pred),'CV_runtime':stats.fit_time[1]+stats.score_time[1],
    'CV_score':stats.test_score[1],'CV_std':stats.test_score[2]},ignore_index=True)

from sklearn.cross_decomposition import PLSRegression
pls = PLSRegression(n_components=20)
t0 = time()
pls.fit(x_train,y_train)
pred = pls.predict(x_test).flatten()
t1 = time()
stats = pd.DataFrame(cross_validate(pls,x_shuffle,y_shuffle,cv=cv)).describe()
result = result.append({'model':'PLS','runtime':(t1-t0),'RMSE':rms(y_test,pred),
    'R2':metrics.r2_score(y_test,pred),'CV_runtime':stats.fit_time[1]+stats.score_time[1],
    'CV_score':stats.test_score[1],'CV_std':stats.test_score[2]},ignore_index=True)

from sklearn.neural_network import MLPRegressor
# nn3 = MLPRegressor(hidden_layer_sizes=(200,200,100,100),activation='relu',alpha=0.07,learning_rate='constant')
nn3 = MLPRegressor(hidden_layer_sizes=(50,50,50,50),activation='relu',alpha=0.1,learning_rate='constant',random_state=100)
t0 = time()
nn3.fit(x_train, y_train)
pred = nn3.predict(x_test) 
t1 = time()
stats = pd.DataFrame(cross_validate(nn3,x_shuffle,y_shuffle,cv=cv)).describe()
result = result.append({'model':'MLPR','runtime':(t1-t0),'RMSE':rms(y_test,pred),
    'R2':metrics.r2_score(y_test,pred),'CV_runtime':stats.fit_time[1]+stats.score_time[1],
    'CV_score':stats.test_score[1],'CV_std':stats.test_score[2]},ignore_index=True)

result

In [ ]:
pd.concat([result,result.rank(axis=0)],axis=1).to_csv("/Users/kenny/Library/CloudStorage/OneDrive-ACCUREBatteryIntelligenceGmbH/Thesis/data/results_lab.csv")

## Visual Comparison

In [ ]:
result = pd.DataFrame()
models = [lr,poly,gpr,svr,nn3]
# models = [lr,ridge,lasso,en,poly]
# test_id = ids
test_id = [234,163,155,167,200,230]
num_x = 2
num_y = 3
fig,ax = plt.subplots(num_y,num_x,figsize=(8,5*num_y),constrained_layout=True)
fig.suptitle("Dataset Delta Test Set")
for i in range(num_y):
    for j in range(num_x):
        # n = i*num_x + j
        id = test_id[i*num_x+j]
        data = df[df["test_id"]==id]
        actual = data['soh'].values
        period = data.shape[0]-16
        input = data.iloc[period:].drop(["test_id","dsoh","soh"],axis=1).values
        input = sc.transform(input)    
        ax[i,j].plot(actual,label='actual')
        for model in models:
            y = model.predict(input).flatten()
            pred_soh = actual[0:period]
            for q in range(0,len(y)):
                pred_soh = np.append(pred_soh,pred_soh[-1]+y[q])
            rmse_d = rms(data['dsoh'][period:],y)
            rmse_s = rms(actual,pred_soh)
            result = result.append(
                {"model":model.__class__.__name__,"RMSE_target":rmse_d,"RMSE_soh":rmse_s,"Final_diff":(np.abs(actual[-1]-pred_soh[-1]))},ignore_index=True)
            ax[i,j].plot(np.arange(period-1,len(pred_soh)),pred_soh[period-1:],'s-',label=str(model))
        ax[i,j].set_ylabel("SOH (%)")
        ax[i,j].set_xlabel("Age (Day)")
        ax[i,j].set_title(f"Test ID: {id}")
        ax[i,j].legend(["Actual","Linear","Polynomial","GPR","SVR","MLPR"])
    # plt.legend()
display(result.groupby('model').mean())
# result.to_csv(f"/Users/kenny/Library/CloudStorage/OneDrive-ACCUREBatteryIntelligenceGmbH/Thesis/data/test_lab.csv")

In [ ]:
# all test set
result = pd.DataFrame()
models = [lr,poly,gpr,svr,nn3]
# test_id = ids
test_id = verif_id
n = 18
for index,id in enumerate(test_id):
    fig,ax = plt.subplots(figsize=(8,8))
    data = df[df["test_id"]==id]
    actual = data['soh'].values
    # period = data.shape[0]-n
    period = 6
    input = data.iloc[period:].drop(["test_id","dsoh","soh"],axis=1).values
    input = sc.transform(input)
    ax.plot(actual)
    for m in models:
        model = m
        y  = model.predict(input).flatten()
        pred_soh = actual[0:period]
        for i in range(0,len(y)):
            pred_soh = np.append(pred_soh,pred_soh[-1]+y[i])
        rmse_d = rms(data['dsoh'][period:],y)
        rmse_s = rms(actual,pred_soh)
        result = result.append(
            {"model":model.__class__.__name__,"RMSE_target":rmse_d,"RMSE_soh":rmse_s,"Final_diff":(np.abs(actual[-1]-pred_soh[-1]))},ignore_index=True)
        ax.plot(np.arange(period-1,len(pred_soh)),pred_soh[period-1:],'^-')
    plt.ylabel("SOH (%)")
    plt.xlabel("Age (Month)")
    # plt.title(f"Index {index} ID:{id}")
    plt.title(f"Test ID:{id}")
    plt.legend(["Actual","Linear","Polynomial","GPR","SVR","MLPR"])
result = result.groupby('model').mean()
display(result)
pd.concat([result,result.rank(axis=0)],axis=1).to_csv(f"/Users/kenny/Library/CloudStorage/OneDrive-ACCUREBatteryIntelligenceGmbH/Thesis/data/test_lab.csv")